### stage_model_regressor

predicting target: end_rank = start_rank + change(old target)

base: 14./stage_model_regressor_withneighbor-newfeatures

prediction models of chg_of_rank_in_stage on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pickle

In [2]:
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.linear_model.ridge import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm.classes import SVR
from sklearn.svm import LinearSVR
from sklearn.utils import shuffle
from sklearn import metrics
import xgboost as xgb

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.ridge module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

In [3]:
# bulid regression model
regressors = ['currank','avgrank','dice','lasso','ridge','rf',
              'svr_rbf','svr_lin','xgb']
              #'svr_rbf','svr_lin','svr_poly','xgb']
    
train_x, train_y, test_x, test_y = None, None, None, None

def get_regressor(regressor = 'lr'):
    if regressor == "lasso":
        clf = LassoCV(cv=5, random_state=0)
    elif regressor == "ridge":
        clf = RidgeCV(alphas=np.logspace(-6, 6, 13))
    elif regressor == "rf":
        #clf = RandomForestRegressor(n_estimators=100)
        clf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=12, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
    elif regressor == 'svr_rbf':
        clf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
    elif regressor == 'svr_lin':
        #clf = SVR(kernel='linear', C=100, gamma='auto')
        clf = SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
            kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
        #clf = SVR(kernel='linear', C=1, gamma='auto')
        #clf = LinearSVR(random_state=0, tol=1e-5)
    elif regressor == 'svr_poly':
        clf = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1)
    elif regressor == 'xgb':
        #clf = xgb.XGBRegressor(objective="reg:linear", random_state=42, max_depth=3)
        clf = xgb.XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0.3, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=7,
             min_child_weight=4, monotone_constraints=None,
             n_estimators=100, n_jobs=1, nthread=1, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8, tree_method=None,
             validate_parameters=False, verbosity=None)
    elif regressor == 'dice':
        clf = RandomDice('1234')
    elif regressor == 'currank':
        clf = CurRank()
    elif regressor == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        #pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        pred_y_avg = pred_y
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    mae = metrics.mean_absolute_error(test_y, pred_y) 
    rmse = math.sqrt(metrics.mean_squared_error(test_y, pred_y))
    r2 = metrics.r2_score(test_y, pred_y)
    print('rmse=%.2f, mae=%.2f, r2=%.2f'%(rmse, mae, r2))
    return mae,rmse, r2
    
#
#features
#    cols=[Myidx, 'target','start_lap', eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
#
#  target id = 1
#  start id = 2 by default

def split_by_eventid(stagedata, train_events, test_events, startid=2):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'].isin(train_events)].to_numpy()
    test  = stagedata[stagedata['eventid'].isin(test_events)].to_numpy()

    #2:car_number
    train_x = train[:,startid:]
    #train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    train_y = train[:,1]
    test_x = test[:,startid:]
    #test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    test_y = test[:,1]
    
    #change target to endrank
    #train_y = train_y + train[:,8] 
    #test_y = test_y + test[:,8]
    
    train = stagedata[stagedata['eventid'].isin(train_events)]
    test  = stagedata[stagedata['eventid'].isin(test_events)]
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    #train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    train_y = train[:,1]
    test_x = test[:,2:]
    #test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    test_y = test[:,1]
    
    return train, test, train_x, train_y, test_x, test_y


def regressor_model(name='svr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_regressor(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    
    #int only
    pred_y = pred_y.astype(int)

    
    score = evaluate(test_y, pred_y)
    return score, pred_y

In [4]:
def build_df_noendlap(testdf, pred_y):
    """
    build a standard stint prediction result:
    carno	startlap	startrank	endrank	diff	sign	pred_endrank	pred_diff	pred_sign
    endlap pred_endlap
    """
    
    print('build_df: len testdf=%d, len of pred_y=%d'%(len(testdf), len(pred_y)))
    
    test = testdf[['car_number','start_lap','start_rank','target']].values
    test[:,1] = test[:,1]-1
    test[:,2] = test[:,2]-1
    test[:,3] = test[:,2] + test[:,3]
    dfout = pd.DataFrame(test, columns =['carno','startlap', 'startrank','endrank'])
    
    dfout['diff'] = dfout['endrank'] - dfout['startrank']
    signVec = dfout['diff'].values.copy()
    for idx in range(len(signVec)):
        sign = 0
        if signVec[idx] > 0:
            sign = 1
        elif signVec[idx] < 0:
            sign = -1
        signVec[idx] = sign
    dfout['sign'] = signVec

    # add predictions
    dfout['pred_endrank'] = pred_y +  dfout['startrank']
    dfout['pred_diff'] = pred_y 
    signVec = dfout['pred_diff'].values.copy()
    for idx in range(len(signVec)):
        sign = 0
        if signVec[idx] > 0:
            sign = 1
        elif signVec[idx] < 0:
            sign = -1
        signVec[idx] = sign
    dfout['pred_sign'] = signVec
    return dfout
    
def build_df(testdf, pred_y):
    """
    build a standard stint prediction result:
    carno	startlap	startrank	endrank	diff	sign	pred_endrank	pred_diff	pred_sign
    endlap pred_endlap
    """
    
    print('build_df: len testdf=%d, len of pred_y=%d'%(len(testdf), len(pred_y)))
    
    test = testdf[['car_number','start_lap','start_rank','target','stint_len']].values
    test[:,1] = test[:,1]-1
    test[:,2] = test[:,2]-1
    test[:,3] = test[:,2] + test[:,3]
    #endlap
    test[:,4] = test[:,4] + test[:,1]
    dfout = pd.DataFrame(test, columns =['carno','startlap', 'startrank','endrank','endlap'])
    dfout['pred_endlap'] = 0
    
    dfout['diff'] = dfout['endrank'] - dfout['startrank']
    signVec = dfout['diff'].values.copy()
    for idx in range(len(signVec)):
        sign = 0
        if signVec[idx] > 0:
            sign = 1
        elif signVec[idx] < 0:
            sign = -1
        signVec[idx] = sign
    dfout['sign'] = signVec

    # add predictions
    dfout['pred_endrank'] = pred_y +  dfout['startrank']
    dfout['pred_diff'] = pred_y 
    signVec = dfout['pred_diff'].values.copy()
    for idx in range(len(signVec)):
        sign = 0
        if signVec[idx] > 0:
            sign = 1
        elif signVec[idx] < 0:
            sign = -1
        signVec[idx] = sign
    dfout['pred_sign'] = signVec
    return dfout

def test_cv():
    global train_x, train_y, test_x, test_y
    
    cols = ['runid','trainsize','testsize','testdistribution']
    cols.extend(regressors)
    print('cols:%s'%cols)
    retdf0 = pd.DataFrame([],columns=cols)
    retdf1 = pd.DataFrame([],columns=cols)

    events = set(stagedata['eventid'])

    years = ['2013','2014','2015','2016','2017','2018','2019']
    #events = ['Indy500']
    eventsname = [f'Indy500-{x}' for x in years]
    events_id={key:idx for idx, key in enumerate(eventsname)}
    for eventid in events:
        print('Testset = %s'%eventsname[eventid])

        train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
        test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
        #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
        #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

        #record
        rec0 = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
        rec1 = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

        pred_y = [0 for x in range(len(regressors))]
        acc0 = [0 for x in range(len(regressors))]
        acc1 = [0 for x in range(len(regressors))]
        for idx, clf in enumerate(regressors):
            acc, pred_y[idx] = regressor_model(clf)
            acc0[idx] = acc[0]
            acc1[idx] = acc[2]

        rec0.extend(acc0)
        rec1.extend(acc1)
        #print('rec:%s'%rec)

        #new df
        df = pd.DataFrame([rec0],columns=cols)
        retdf0 = pd.concat([retdf0, df])        

        df = pd.DataFrame([rec1],columns=cols)
        retdf1 = pd.concat([retdf1, df])        


    #retdf0.to_csv('regressors_stagedata_splitbyevent%s_rmse.csv'%suffix)
    #retdf1.to_csv('regressors_stagedata_splitbyevent%s_r2.csv'%suffix)
    #retdf0.to_csv('crossvalid_stagedata_regressor_%s.csv'%suffix, float_format='%.3f')

    df_event_rmse = retdf0
    df_event_r2 = retdf1
    return df_event_rmse
    
def test_20182019(startid = 2):
    global train_x, train_y, test_x, test_y

    pred_df = {}
    
    ### train 2013-2017
    #load data
    cols = ['runid','trainsize','testsize','testdistribution']
    cols.extend(regressors)
    print('cols:%s'%cols)
    retdf0 = pd.DataFrame([],columns=cols)
    retdf1 = pd.DataFrame([],columns=cols)

    eventsids = set(stagedata['eventid'])

    #for event in events:
    for event in _test_events:
        test_eventid = events_id[event]
        
        if test_eventid in _train_events:
            continue
        
        print('Testset = %s'% event, test_eventid)

        traindf, testdf, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, 
                                                            _train_events, [test_eventid], startid=startid)
        test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
        #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
        #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

        try:
            #record
            rec0 = [event,train_x.shape[0],test_x.shape[0],test_distribution]
            rec1 = [event,train_x.shape[0],test_x.shape[0],test_distribution]

            pred_y = [0 for x in range(len(regressors))]
            acc0 = [0 for x in range(len(regressors))]
            acc1 = [0 for x in range(len(regressors))]
            for idx, clf in enumerate(regressors):
                acc, pred_y[idx] = regressor_model(clf)
                acc0[idx] = acc[0]
                acc1[idx] = acc[2]

                #build pred df
                if not event in pred_df:
                    pred_df[event] = {}
                
                #pred_df[event][clf] = build_df(testdf, pred_y[idx])
                
                testdf_full = stagedata_full[stagedata_full['eventid'].isin([test_eventid])]
                pred_df[event][clf] = build_df(testdf_full, pred_y[idx])


            rec0.extend(acc0)
            rec1.extend(acc1)
            #print('rec:%s'%rec)
        except:
            print('ERROR in ', event)
            continue

        #new df
        df = pd.DataFrame([rec0],columns=cols)
        retdf0 = pd.concat([retdf0, df])        

    return retdf0, pred_df   

### test oracle with stint_len

In [5]:
_inlap_status = 0
_featureCnt = 9
#
# input data parameters
#
# event -> car#, maxlap
_race_info = {}
# the races have 7 years data 
races = ['Indy500', 'Texas','Iowa','Pocono']
years = ['2013','2014','2015','2016','2017','2018','2019']

events = []
for race in races:
    events.extend([f'{race}-{x}' for x in years])

events.extend(['Phoenix-2018','Gateway-2018','Gateway-2019'])
events_id={key:idx for idx, key in enumerate(events)}

# dataset shared
dataOutputRoot = "data/"
covergap = 1
dbid = f'IndyCar_d{len(events)}_v{_featureCnt}_p{_inlap_status}'
LAPTIME_DATASET = f'{dataOutputRoot}/laptime_rank_timediff_pit-oracle-{dbid}.pickle' 
STAGE_DATASET = f'{dataOutputRoot}/stagedata-{dbid}.pickle' 
PITCOVERED_DATASET = f'{dataOutputRoot}/pitcoveredlaps-{dbid}-g{covergap}.pickle'
PITSTOP_DATASET = f'{dataOutputRoot}/pitstop-{dbid}.csv' 
    
print('Load laptime and stage dataset:',LAPTIME_DATASET, STAGE_DATASET)
with open(LAPTIME_DATASET, 'rb') as f:
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 
with open(STAGE_DATASET, 'rb') as f:
    #stagedata = pickle.load(f, encoding='latin1') 
    stagedata_o, _race_info, _events, _events_id = pickle.load(f, encoding='latin1') 
pitstop_df = pd.read_csv(PITSTOP_DATASET)

#check it
if not _events == events:
    print('Error, events mismatch at:', STAGE_DATASET)
    sys.exit(-1)

#trainrace = 'Indy500','Iowa','Pocono','Texas'
trainrace = 'Pocono'
    
#_train_events = [events_id[x] for x in [f'{trainrace}-{x}' for x in ['2013','2014','2015','2016','2017']]]
#_test_events = [f'{trainrace}-{x}' for x in ['2018','2019']]

if trainrace == 'Pocono':
    # 2014, 2019 is bad
    _train_years = ['2013','2015','2016','2017']
    _test_events = [f'{trainrace}-2018']
else:
    _train_years = ['2013','2014','2015','2016','2017']
    _test_events = [f'{trainrace}-2018',f'{trainrace}-2019']
    
_train_events = [events_id[x] for x in [f'{trainrace}-{x}' for x in _train_years]]

Load laptime and stage dataset: data//laptime_rank_timediff_pit-oracle-IndyCar_d31_v9_p0.pickle data//stagedata-IndyCar_d31_v9_p0.pickle


In [6]:
_trim = 0
_include_final = True
_include_stintlen = True
#_include_stintlen = False

include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''

STINT_DATASET= f'{dataOutputRoot}/stage-IndyCar-d{len(events)}-end{include_str}{stint_str}.csv'

stagedata = pd.read_csv(STINT_DATASET)
stagedata.fillna(0, inplace=True)
stagedata.info()
stagedata_raw = stagedata.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4179 entries, 0 to 4178
Data columns (total 39 columns):
Unnamed: 0                   4179 non-null int64
target                       4179 non-null int64
start_lap                    4179 non-null int64
eventid                      4179 non-null int64
car_number                   4179 non-null int64
stageid                      4179 non-null int64
firststage                   4179 non-null int64
pit_in_caution               4179 non-null int64
start_position               4179 non-null int64
start_rank                   4179 non-null int64
start_rank_ratio             4179 non-null float64
top_pack                     4179 non-null int64
bottom_pack                  4179 non-null int64
average_rank                 4179 non-null float64
average_rank_all             4179 non-null float64
change_in_rank               4179 non-null int64
change_in_rank_all           4179 non-null float64
rate_of_change               4179 non-null int64
rat

In [7]:
stagedata = stagedata_raw.copy()
stagedf = stagedata
stagedata_full = stagedata_raw.copy()    
stagedf[(stagedf['car_number']==12) & (stagedf['eventid']==5)]

,Unnamed: 0,target,start_lap,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,...,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank,cautionlaps_all,cautionlaps_cur,stint_len
1156,1327,2,32,5,12,1,1,0,3,4,...,66.10705,-1,0,0,0,-3,-5,0,0,18
1157,1328,-5,50,5,12,2,1,1,3,6,...,61.85245,-6,-5,0,6,3,3,2,2,44
1158,1329,1,94,5,12,3,1,0,3,1,...,117.01525,0,0,0,-3,3,3,15,0,35
1159,1330,6,129,5,12,4,1,0,3,2,...,59.24225,-3,0,0,2,1,0,0,0,42
1160,1331,-7,171,5,12,5,1,0,3,8,...,59.50010,5,2,6,4,-6,5,20,0,29


In [8]:
#df_event_oracle = test_cv()

In [9]:
retdf, preddf = {}, {}

In [10]:
#retdf['oracle'], preddf['oracle'] = test_20182019()
#retdf['oracle-start5'], preddf['oracle-start5'] = test_20182019(startid=5)

In [11]:
stagedata = stagedata.drop(['cautionlaps_all','cautionlaps_cur'],axis=1)
#retdf['oracle-start5-nonewtry'], preddf['oracle-start5-nonewtry'] = test_20182019(startid=5)
retdf['oracle-start2-nonewtry'], preddf['oracle-start2-nonewtry'] = test_20182019(startid=2)

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lasso', 'ridge', 'rf', 'svr_rbf', 'svr_lin', 'xgb']
Testset = Pocono-2018 26
[*] predict with currank model
rmse=2.82, mae=1.72, r2=-0.01
build_df: len testdf=101, len of pred_y=101
[*] predict with avgrank model
rmse=4.52, mae=3.06, r2=-1.59
build_df: len testdf=101, len of pred_y=101
[*] predict with dice model
rmse=5.77, mae=4.02, r2=-3.22
build_df: len testdf=101, len of pred_y=101
[*] predict with lasso model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.667400415508382, tolerance: 1.0896556792873053
  tol, rng, random, positive)


rmse=41.16, mae=12.51, r2=-214.13
build_df: len testdf=101, len of pred_y=101
[*] predict with ridge model
rmse=67.86, mae=42.14, r2=-583.81
build_df: len testdf=101, len of pred_y=101
[*] predict with rf model
rmse=2.47, mae=1.79, r2=0.23
build_df: len testdf=101, len of pred_y=101
[*] predict with svr_rbf model
rmse=2.82, mae=1.72, r2=-0.01
build_df: len testdf=101, len of pred_y=101
[*] predict with svr_lin model
rmse=54.44, mae=38.07, r2=-375.34
build_df: len testdf=101, len of pred_y=101
[*] predict with xgb model
rmse=3.12, mae=2.37, r2=-0.24
build_df: len testdf=101, len of pred_y=101


### test without stint_len

In [12]:
stagedata = stagedata_raw.copy()    
stagedata_full = stagedata_raw.copy()    

In [13]:
stagedata = stagedata.drop(['stint_len'],axis=1)

In [14]:
stagedf = stagedata
stagedf[(stagedf['car_number']==12) & (stagedf['eventid']==5)]

,Unnamed: 0,target,start_lap,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,...,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank,cautionlaps_all,cautionlaps_cur
1156,1327,2,32,5,12,1,1,0,3,4,...,32,66.10705,-1,0,0,0,-3,-5,0,0
1157,1328,-5,50,5,12,2,1,1,3,6,...,18,61.85245,-6,-5,0,6,3,3,2,2
1158,1329,1,94,5,12,3,1,0,3,1,...,44,117.01525,0,0,0,-3,3,3,15,0
1159,1330,6,129,5,12,4,1,0,3,2,...,35,59.24225,-3,0,0,2,1,0,0,0
1160,1331,-7,171,5,12,5,1,0,3,8,...,42,59.50010,5,2,6,4,-6,5,20,0


In [15]:
#df_event = test_cv()

In [16]:
#df_event

In [17]:
#retdf['normal'], preddf['normal'] = test_20182019()
#retdf['normal-start5'], preddf['normal-start5'] = test_20182019(startid=5)

In [18]:
stagedata = stagedata.drop(['cautionlaps_all','cautionlaps_cur'],axis=1)
#retdf['normal-start5-nonewtry'], preddf['normal-start5-nonewtry'] = test_20182019(startid=5)
retdf['normal-start2-nonewtry'], preddf['normal-start2-nonewtry'] = test_20182019(startid=2)

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lasso', 'ridge', 'rf', 'svr_rbf', 'svr_lin', 'xgb']
Testset = Pocono-2018 26
[*] predict with currank model
rmse=2.82, mae=1.72, r2=-0.01
build_df: len testdf=101, len of pred_y=101
[*] predict with avgrank model
rmse=4.52, mae=3.06, r2=-1.59
build_df: len testdf=101, len of pred_y=101
[*] predict with dice model
rmse=5.77, mae=4.02, r2=-3.22
build_df: len testdf=101, len of pred_y=101
[*] predict with lasso model
rmse=39.61, mae=12.95, r2=-198.27
build_df: len testdf=101, len of pred_y=101
[*] predict with ridge model
rmse=56.69, mae=39.30, r2=-407.09
build_df: len testdf=101, len of pred_y=101
[*] predict with rf model
rmse=2.54, mae=1.76, r2=0.18
build_df: len testdf=101, len of pred_y=101
[*] predict with svr_rbf model
rmse=2.82, mae=1.72, r2=-0.01
build_df: len testdf=101, len of pred_y=101
[*] predict with svr_lin model
rmse=29.02, mae=23.58, r2=-105.92
build_df: len testdf=101, len of pred

In [19]:
for key in retdf:
    print(key)
    print(retdf[key])

oracle-start2-nonewtry
         runid trainsize testsize testdistribution   currank   avgrank  \
0  Pocono-2018       561      101   +:34,0:30,-:37  1.722772  3.059406   

       dice      lasso      ridge        rf   svr_rbf    svr_lin       xgb  
0  4.019802  12.514851  42.138614  1.792079  1.722772  38.069307  2.366337  
normal-start2-nonewtry
         runid trainsize testsize testdistribution   currank   avgrank  \
0  Pocono-2018       561      101   +:34,0:30,-:37  1.722772  3.059406   

       dice      lasso     ridge        rf   svr_rbf    svr_lin       xgb  
0  4.019802  12.950495  39.29703  1.762376  1.722772  23.584158  2.732673  


### save result df

In [20]:
import pickle
def save_result(dfs, datafile):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [dfs]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

In [21]:

version = f'IndyCar-d{trainrace}-endlap'

#outfile=f'{dataroot}/stint-dfout-mlmodels-{version}-end{include_str}-normal-t{_trim}-alldata.pickle'
#save_result(preddf, outfile)


In [22]:
#output_file = f'stage-indy500-2013-2019-end{include_str}{stint_str}-t{_trim}.csv'
#outfile=f'{dataroot}/stint-dfout-mlmodels-{version}-end{include_str}-normal-t{_trim}-newtry-tuned.pickle'
#save_result(preddf['normal'], outfile)
#outfile=f'{dataroot}/stint-dfout-mlmodels-{version}-end{include_str}-oracle-t{_trim}-newtry-tuned.pickle'
#save_result(preddf['oracle'], outfile)

outfile=f'{dataOutputRoot}/stint-dfout-mlmodels-{version}-end{include_str}-normal-t{_trim}-tuned.pickle'
save_result(preddf['normal-start2-nonewtry'], outfile)
outfile=f'{dataOutputRoot}/stint-dfout-mlmodels-{version}-end{include_str}-oracle-t{_trim}-tuned.pickle'
save_result(preddf['oracle-start2-nonewtry'], outfile)


In [23]:
preddf.keys()

dict_keys(['oracle-start2-nonewtry', 'normal-start2-nonewtry'])